In [ ]:
import xmltodict

In [ ]:
with open("./STM32F7_svd/STM32F7_svd_V1.4/STM32F7x6.svd",'r') as xf:
    desc = xmltodict.parse(xf.read())

In [ ]:
def prnt(item, indent):
    if hasattr(item,'prnt'):
        return item.prnt(indent)
    elif type(item) is dict:
        return"".join([
            (" "*indent) +name+": "+prnt(subitem, indent+2)+"\n" for name,subitem in item.items()
        ])
    else:
        return str(item)
        


class SVDElement:
    def __init__(self, dic, parse_table):
        for name,value in dic.items():
            if type(name) is str:
                self.__dict__[name.replace('@','')]=parse_table.get(name, lambda s,p : str(s) )(value, parse_table)
    
    def prnt(self, indent=0):
        return"".join([(" "*indent) +name+": "+prnt(item, indent+2)+'\n' for name,item in self.__dict__.items()])
        
    def __repr__(self):
        return self.prnt()

class Field(SVDElement):
    def __init__(self, field, parse_table):
        super(Field, self).__init__(field, parse_table)
        
    def to_cpp(self, reg_type):
        start = self.bitOffset
        stop = start + self.bitWidth - 1
        if start == stop:
            return f"bitfield_t<{reg_type},{start}> {self.name};\n"
        return f"bitfield_t<{reg_type},{start},{stop}> {self.name};\n"

class Register(SVDElement):
    def __init__(self, reg, parse_table):
        super(Register, self).__init__(reg, parse_table)
        
    def __eq__(self, other):
        return self.name == other.name
    
    def to_cpp(self):
        name = self.name
        reg_type = self.name+"_t"
        offset = self.addressOffset
        if self.size == 32:
            str_type = 'uint32_t'
        elif self.size == 16:
            str_type = 'uint16_t'
        return f"""struct {reg_type}:reg_t<{str_type}, base_address+{offset}>
{{
    {"".join([bitfield.to_cpp(reg_type) for bitfield in self.fields.values()])} }}{self.name};"""+"\n"
        
class Peripheral(SVDElement):
    def __init__(self,periph, parse_table):
        super(Peripheral, self).__init__(periph, parse_table)
    def __eq__(self, other):
        return self.registers.keys() == other.registers.keys()
    
    def to_cpp(self):
        return f"""
template<uint32_t base_address>
struct {self.name}_t
{{
    {"".join([register.to_cpp() for register in self.registers.values()])}
}};
            """+"\n"
        
class Device(SVDElement):
    def __init__(self,dev , parse_table):
        super(Device, self).__init__(dev, parse_table)
        
_ctors_parse_table={
    'field': Field,
    'register':Register,
    'peripheral':Peripheral,
}

def listify(item_or_list):
    if type(item_or_list) is list:
        return item_or_list
    else:
        return [item_or_list]

def ctor(key, dic, parse_table):
    return _ctors_parse_table[key](dic[key], parse_table)

_parse_table={
    'addressOffset':lambda s,p : int(s,16),
    'bitOffset':lambda s,p : int(s),
    'bitWidth':lambda s,p : int(s),
    'size': lambda s,p  : int(s,16),
    'resetValue': lambda s,p : int(s,16),
    'fields': lambda item,parse_table : { field['name']:Field(field,parse_table) for field in listify(item["field"])},
    'registers': lambda item,parse_table : { register['name']:Register(register,parse_table) for register in listify(item["register"])},
    'peripherals': lambda item,parse_table : { peripheral['name']:Peripheral(peripheral,parse_table) for peripheral in listify(item["peripheral"])},
}

d = Device(desc['device'],_parse_table)
for p in d.peripherals.values():
    if hasattr(p,'derivedFrom'):
        p.__dict__['registers']=d.peripherals[p.derivedFrom].registers

In [ ]:
def all_same(device, periph_name):
    periphs = [p for n,p in device.peripherals.items() if periph_name in n]
    return all([[periph]*len(periphs)==periphs for periph in periphs])

all_same(d,'GPIO')
all_same(d,'SPI')

In [ ]:
d.peripherals.keys()

In [ ]:
rcc = d.peripherals["RCC"].to_cpp()
with open('rcc.hpp', 'w') as f:
    f.write(f"""
#pragma once
#include <stdint.h>
#include "../register.hpp"


using namespace ucpp::registers;
namespace ucpp::stm32::rcc {{
{rcc}
}}
    """)

In [ ]:
d.peripherals["GPIOB"]